In [27]:
from pysr import PySRRegressor, TemplateExpressionSpec

import matplotlib.pyplot as plt
import numpy as np

import camb

from scipy.optimize import curve_fit

In [28]:
# Load Data
pars     = np.load('CL_data/parameter_test9.npy')  # [H0, ombh2, omch2 ] x 100
lensed   = np.load('CL_data/lensed_CL9.npy')     # [C_2, ..., C_5000] x 100 (lensed)
unlensed = np.load('CL_data/unlensed_CL9.npy')     # [C_2, ..., C_5000] x 100 (unlensed)

In [29]:
ombs = pars[:,1]
omcs = pars[:,2]

ells = np.array([l for l in range(2, 5000)])

In [30]:
lensing = lensed/unlensed
camb_data_format = []
for i in range(len(lensing)):
    for j in range(len(lensing[i])):
        camb_data_format.append(lensing[i][j])
#lensing = [lensing[:][i][0] for i in range(len(lensing))]
#lensing

In [31]:
def get_lensing_camb(omb, omc):
    pars = camb.set_params(H0 = 67.4, ombh2 = omb, omch2 = omc, lmax=5000)
    results = camb.get_results(pars)
    powers = results.get_cmb_power_spectra(pars, CMB_unit='muK')
    lensed = powers['total'][2:5000, 0]
    unlensed = powers['unlensed_total'][2:5000, 0]
    return lensed/unlensed

In [32]:
def return_error(func):
    return [100*np.abs(func(ells, pars_pysr[ind, 1], pars_pysr[ind, 2]) - lensed[ind]/unlensed[ind])/(lensed[ind]/unlensed[ind]) for ind in range(0, 100)]

In [33]:
fid_ombh2 = 0.0224
fid_omch2 = 0.12

In [34]:
def func(X, shift1, shift2):
    ell = X[0]
    ommh2 = X[1]
    #ell, ommh2 = X
    beta3 = 3776 * (1 + shift1 * ommh2/0.1424)
    beta4 = 341 * (1 + shift2 * ommh2/0.1424)
    alpha = (ommh2/0.1414)/0.4
    sigma = (1 + np.exp(-(ell - beta3)/beta4))**-1
    poly = (0.7 * (ell/2800)**alpha -1)
    
    return (poly*sigma + 1).flatten()

In [35]:
#ombs = [0.02, 0.022, 0.022, 0.022, 0.022, 0.022]
#omcs = [0.1, 0.1, 0.12, 0.15, 0.18, 0.19]
cosmos = [ombs[i] + omcs[i] for i in range(len(ombs))]
n = len(cosmos) #number of cosmologies
params = np.zeros((len(ells) * n, 2))
for i in range(n):
    params[(i)*len(ells):(i+1)*len(ells),0] = ells
for i in range(n):
    params[(i)*len(ells):(i+1)*len(ells),1] = cosmos[i]

In [36]:
#camb_data = [get_lensing_camb(ombs[i], omcs[i]) for i in range(len(ombs))]

In [37]:
#camb_data_format = np.reshape(camb_data,(len(camb_data)*len(camb_data[0]), 1))
#camb_data_format = [camb_data_format[i][0] for i in range(len(camb_data_format))]

In [38]:
p0 = 0,0

In [39]:
x1 = params[:,0]
x2 = params[:,1]

In [40]:
len(camb_data_format)

2998800

In [41]:
answers, _ = curve_fit(func,(x1, x2),camb_data_format, p0)

In [42]:
answers

array([-0.01105373,  0.41824394])